# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Read csv cities file and store into data frame:
cities = pd.read_csv("../output_data/cities.csv")
cities.head()

,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Isabela,PH,1610622816,6.7085,121.9711,40,78,82.40,3.44
1,Lebu,CL,1610622816,-37.6167,-73.6500,4,85,52.77,13.02
2,Hermanus,ZA,1610622817,-34.4187,19.2345,27,56,90.00,1.01
3,Jamestown,US,1610622804,42.0970,-79.2353,90,80,37.00,8.05
4,Saint Paul Harbor,US,1610622817,57.7900,-152.4072,90,75,37.40,17.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Max humidity in cities dataframe
humidity = cities["Humidity (%)"].astype(float)
maxhumidity = humidity.max()

#Store Lat and Lng into locations
locations = cities[["Lat", "Lng"]]

In [5]:
#Humidity heat map layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the dataframe to find ideal weather condition
hotel_df = cities.loc[(cities["Max Temp (F)"] > 70) & (cities["Max Temp (F)"] < 80) & (cities["Wind Speed (mph)"] < 10) & (cities["Cloudiness (%)"] == 0), :]
hotel_df = hotel_df.dropna(how='any')
hotel_df.reset_index(inplace=True)
del hotel_df['index']
hotel_df.head()

,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Gympie,AU,1610622600,-26.1898,152.6650,0,85,73.00,2.48
1,Nam Som,TH,1610622840,17.7704,102.1895,0,47,78.01,4.61
2,Riyadh,SA,1610622749,24.6877,46.7219,0,18,71.60,5.75
3,Ballina,AU,1610622934,-28.8667,153.5667,0,78,73.40,6.91
4,Phan Thiet,VN,1610622923,10.9333,108.1000,0,79,71.64,8.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Hotel Name
0,Gympie,AU,1610622600,-26.1898,152.6650,0,85,73.00,2.48,
1,Nam Som,TH,1610622840,17.7704,102.1895,0,47,78.01,4.61,
2,Riyadh,SA,1610622749,24.6877,46.7219,0,18,71.60,5.75,
3,Ballina,AU,1610622934,-28.8667,153.5667,0,78,73.40,6.91,
4,Phan Thiet,VN,1610622923,10.9333,108.1000,0,79,71.64,8.61,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
#assign marker layer to variable
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))